In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - CIFAR10

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'cifar10'
dataset_percentage = '5'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for cifar10_5...
Found 1750 images belonging to 10 classes.
Loading test set for cifar10_5...
Found 500 images belonging to 10 classes.
Loading validation set for cifar10_5...
Found 250 images belonging to 10 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
55/55 [==============================] - 1s 25ms/step - loss: 4.2635 - accuracy: 0.1251 - precision: 0.0158 - recall: 0.0227 - val_loss: 2.1863 - val_accuracy: 0.1440 - val_precision: 0.1250 - val_recall: 0.0333
Epoch 2/50
55/55 [==============================] - 1s 12ms/step - loss: 2.1916 - accuracy: 0.2029 - precision: 0.1545 - recall: 0.0709 - val_loss: 2.3646 - val_accuracy: 0.2240 - val_precision: 0.1875 - val_recall: 0.0562
Epoch 3/50
55/55 [==============================] - 1s 11ms/step - loss: 1.8985 - accuracy: 0.3206 - precision: 0.3409 - recall: 0.1874 - val_loss: 2.2589 - val_accuracy: 0.2600 - val_precision: 0.0937 - val_recall: 0.0729
Epoch 4/50
55/55 [==============================] - 1s 12ms/step - loss: 1.6491 - accuracy: 0.4314 - precision: 0.4551 - recall: 0.2532 - val_loss: 2.3638 - val_accuracy: 0.3060 - val_precision: 0.2812 - val_recall: 0.1208
Epoch 5/50
55/55 [==============================] - 1s 11ms/step - loss: 1.4178 - accuracy: 0.5120 - precisi

Epoch 38/50
55/55 [==============================] - 1s 11ms/step - loss: 9.9597e-04 - accuracy: 1.0000 - precision: 0.9455 - recall: 0.9455 - val_loss: 7.0376 - val_accuracy: 0.2920 - val_precision: 0.3430 - val_recall: 0.2262
Epoch 39/50
55/55 [==============================] - 1s 11ms/step - loss: 9.2371e-04 - accuracy: 1.0000 - precision: 0.9818 - recall: 0.9818 - val_loss: 7.3261 - val_accuracy: 0.2900 - val_precision: 0.2552 - val_recall: 0.2426
Epoch 40/50
55/55 [==============================] - 1s 10ms/step - loss: 8.5795e-04 - accuracy: 1.0000 - precision: 0.9636 - recall: 0.9636 - val_loss: 4.0228 - val_accuracy: 0.2880 - val_precision: 0.2292 - val_recall: 0.2135
Epoch 41/50
55/55 [==============================] - 1s 11ms/step - loss: 7.6840e-04 - accuracy: 1.0000 - precision: 0.9818 - recall: 0.9818 - val_loss: 7.9373 - val_accuracy: 0.2880 - val_precision: 0.4083 - val_recall: 0.3866
Epoch 42/50
55/55 [==============================] - 1s 11ms/step - loss: 7.4596e-04 - a

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,1.413377046585083,2.1863362789154053,0.14399999380111694,0.1249999850988388,0.03333333507180214
2,0.6965498924255371,2.3645782470703125,0.2240000069141388,0.1874999850988388,0.05624999850988388
3,0.6201848983764648,2.2589340209960938,0.25999999046325684,0.0937499925494194,0.0729166567325592
4,0.6908543109893799,2.363809108734131,0.3059999942779541,0.2812499701976776,0.12083333730697632
5,0.6122360229492188,2.231987476348877,0.25600001215934753,0.3895833194255829,0.27916666865348816
6,0.6265809535980225,2.112685203552246,0.31200000643730164,0.3697916567325592,0.2666666507720947
7,0.6347918510437012,2.6247966289520264,0.28600001335144043,0.3385416567325592,0.22708332538604736
8,0.6391627788543701,2.0340278148651123,0.2840000092983246,0.3125,0.21979168057441711
9,0.6010348796844482,2.3435473442077637,0.2800000011920929,0.1979166567325592,0.1197916716337204
10,0.6493551731109619,3.720046281814575,0.301999986171722

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
0.6418325710296631,32.091628551483154,5.955047130584717,0.3199999928474426,0.3916666507720947,0.4464285671710968
